<a href="https://colab.research.google.com/github/KimJisanER/Dacon/blob/main/metabolism_Discriptor_0907_MACCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 !pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

[+] 0.0s
conda-forge/linux-64  ⣾  [+] 0

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import Descriptors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.0 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176723 sha256=77095335cd9e88f11a68fd357b5ddcfa448bcace76cab85ad1e3f567b8be4d08
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred


In [ ]:
def getMolDescriptors(mol, missingVal=None):
    ''' calculate the full list of descriptors for a molecule

        missingVal is used if the descriptor cannot be calculated
    '''
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res


In [ ]:
# load data
train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])

HLM = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train_aug_HLM_0906.csv")

test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])

full = pd.concat([train, test, HLM], axis = 0).reset_index(drop=True)
full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)

In [ ]:
full

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.25900,117.37,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4fc2b20>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.17200,73.47,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4fc2b90>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.58500,62.45,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4fc2c00>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.47500,92.60,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4fc2c70>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.33700,42.43,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4fc2ce0>
...,...,...,...,...,...,...,...,...,...,...,...,...
5573,CHEMBL3286701,Cc1[nH]c2ccccc2c1CCNC(=O)c1ccc(N(C)C)cc1,NaN,17.000,3.510,321.420,2,2,5,3.51482,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4e7c740>
5574,CHEMBL215387,O=C(O)CCCCCCCCCCCNC(=O)NC12CC3CC(CC(C3)C1)C2,NaN,98.000,5.240,392.580,2,3,13,5.24000,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4e7c7b0>
5575,CHEMBL242459,O=C(NC12CC3CC(CC(C3)C1)C2)N[C@H]1CC[C@H](Oc2cc...,NaN,69.000,4.340,412.530,3,3,5,4.34280,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4e7c820>
5576,CHEMBL3735279,CCOC(=O)c1nc(C)c2c(c1N)C(=O)N(Cc1ccccc1)C2=O,NaN,14.000,1.950,339.350,6,1,4,1.94512,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4e7c890>


In [ ]:
from rdkit import DataStructs

allDescrs = [getMolDescriptors(m) for m in full['Molecule']]
full_Descrs = pd.DataFrame(allDescrs)

In [ ]:
fragment_counts = {}

def extract_sp3_carbon_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP3:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_sp3_carbon_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')

# 결과 데이터프레임 출력
print(result_df)

       CC  CCO  cCN   Cc  CCN  CC(C)C   CCC  CN(C)C  cS(N)(=O)=O  CN(C)S  ...  \
0     2.0  2.0  1.0  2.0  0.0     0.0   0.0     0.0          0.0     0.0  ...   
1     1.0  0.0  1.0  2.0  1.0     1.0   0.0     0.0          0.0     0.0  ...   
2     1.0  0.0  0.0  0.0  5.0     0.0   1.0     1.0          0.0     0.0  ...   
3     0.0  0.0  0.0  3.0  4.0     0.0   5.0     1.0          1.0     1.0  ...   
4     1.0  0.0  0.0  1.0  1.0     0.0   0.0     0.0          0.0     0.0  ...   
...   ...  ...  ...  ...  ...     ...   ...     ...          ...     ...  ...   
5573  0.0  0.0  0.0  1.0  1.0     0.0   0.0     0.0          0.0     0.0  ...   
5574  0.0  0.0  0.0  0.0  1.0     3.0  16.0     0.0          0.0     0.0  ...   
5575  0.0  0.0  0.0  0.0  0.0     3.0  10.0     0.0          0.0     0.0  ...   
5576  1.0  1.0  1.0  1.0  0.0     0.0   0.0     0.0          0.0     0.0  ...   
5577  1.0  0.0  1.0  0.0  2.0     0.0   3.0     0.0          0.0     0.0  ...   

      [O-]P   OP  CC[Si]  C

In [ ]:
fragment_counts = {}

def extract_non_ring_sp2_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP2 and atom.IsInRing() == False:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_non_ring_sp2_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df2 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df2.columns = ['non_ring_sp2_' + column for column in result_df2.columns]

# 결과 데이터프레임 출력
print(result_df2)


      non_ring_sp2_COc  non_ring_sp2_CNC  non_ring_sp2_cC(N)=O  \
0                  1.0               1.0                   1.0   
1                  0.0               0.0                   0.0   
2                  0.0               0.0                   0.0   
3                  0.0               0.0                   0.0   
4                  0.0               0.0                   1.0   
...                ...               ...                   ...   
5573               0.0               1.0                   1.0   
5574               0.0               2.0                   0.0   
5575               1.0               2.0                   0.0   
5576               0.0               0.0                   0.0   
5577               0.0               0.0                   1.0   

      non_ring_sp2_C=O  non_ring_sp2_cOC  non_ring_sp2_O=S  non_ring_sp2_c=O  \
0                  1.0               1.0               0.0               0.0   
1                  1.0               0.0       

In [ ]:
fragment_counts = {}

def extract_sp_carbon_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_sp_carbon_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df3 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df3.columns = ['sp_' + column for column in result_df3.columns]

# 결과 데이터프레임 출력
print(result_df3)

      sp_cC#N  sp_C#N  sp_CC#N  sp_C#CN  sp_C#Cc  sp_C#Cn  sp_N#CN  sp_C#CC  \
0         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0   
1         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0   
3         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4         0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0   
...       ...     ...      ...      ...      ...      ...      ...      ...   
5573      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0   
5574      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0   
5575      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0   
5576      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0   
5577      0.0     0.0      0.0      0.0      0.0      0.0      0.0      0.0   

      sp_C#C  sp_[N-]=[N+]=N  
0        0.0        

In [ ]:
fragment_counts = {}

def extract_ring_sp2_fragments(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP2 and atom.IsInRing() == True:
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

# 데이터프레임의 각 행에 대해 반복
for index, row in full.iterrows():
    smiles = row['SMILES']
    fragments = extract_ring_sp2_fragments(smiles)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = full.loc[index, 'SMILES']
    result_data.append(counts)

result_df4 = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
result_df4.columns = ['ring_sp2_' + column for column in result_df4.columns]

# 결과 데이터프레임 출력
print(result_df4)


      ring_sp2_cc(c)O  ring_sp2_ccc  ring_sp2_cc(c)C  ring_sp2_cc(C)[nH]  \
0                 2.0           4.0              1.0                 1.0   
1                 0.0           4.0              0.0                 0.0   
2                 0.0           4.0              0.0                 0.0   
3                 0.0           6.0              1.0                 0.0   
4                 1.0           5.0              2.0                 0.0   
...               ...           ...              ...                 ...   
5573              0.0           8.0              2.0                 1.0   
5574              0.0           0.0              0.0                 0.0   
5575              1.0           4.0              1.0                 0.0   
5576              0.0           5.0              3.0                 0.0   
5577              0.0           8.0              2.0                 0.0   

      ring_sp2_cc(-c)n  ring_sp2_cc(-c)s  ring_sp2_csc  ring_sp2_Cc(n)s  \
0           

In [ ]:
def count_NandO(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    n_count = sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'N')
    o_count = sum(1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'O')
    return n_count + o_count

NandO_data = []

for index, row in full.iterrows():
    smiles = row['SMILES']
    nando_count = count_NandO(smiles)

    if nando_count is not None:
        NandO_data.append({"NandO": nando_count})

NandO_df = pd.DataFrame(NandO_data)

print(NandO_df)


      NandO
0         7
1         4
2         7
3         8
4         4
...     ...
5573      4
5574      5
5575      6
5576      7
5577      6

[5578 rows x 1 columns]


In [ ]:
def count_3_and_4_membered_rings(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None

    in34 = sum(1 for ring in mol.GetRingInfo().AtomRings() if len(ring) in [3, 4])
    in56 = sum(1 for ring in mol.GetRingInfo().AtomRings() if len(ring) in [5, 6])

    return in34, in56

ring_data = []

for index, row in full.iterrows():
    smiles = row['SMILES']
    in34_count, in56_count = count_3_and_4_membered_rings(smiles)

    if in34_count is not None and in56_count is not None:
        ring_data.append({"in34": in34_count, "in56": in56_count})

ring_df = pd.DataFrame(ring_data)

print(ring_df)


      in34  in56
0        0     3
1        0     2
2        0     4
3        0     5
4        0     3
...    ...   ...
5573     0     3
5574     0     4
5575     0     6
5576     0     3
5577     1     3

[5578 rows x 2 columns]


In [ ]:
def count_non_conjugated_atoms(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    noncon_count = 0
    for atom in mol.GetAtoms():
        if not atom.IsInRing() and not atom.GetIsAromatic():
            noncon_count += 1

    return noncon_count

noncon_data = []

for index, row in full.iterrows():
    smiles = row['SMILES']
    noncon_count = count_non_conjugated_atoms(smiles)

    if noncon_count is not None:
        noncon_data.append({"noncon": noncon_count})

noncon_df = pd.DataFrame(noncon_data)

print(noncon_df)


      noncon
0         12
1          5
2          3
3          6
4          4
...      ...
5573       9
5574      18
5575       8
5576      10
5577      14

[5578 rows x 1 columns]


In [ ]:
def calculate_stereocenters(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)
    return len(chiral_centers)

stereocenters_data = []

for index, row in full.iterrows():
    smiles = row['SMILES']
    stereocenters = calculate_stereocenters(smiles)

    if stereocenters is not None:
        stereocenters_data.append({"stereocenters": stereocenters})

stereocenters_df = pd.DataFrame(stereocenters_data)

print(stereocenters_df)


      stereocenters
0                 0
1                 1
2                 0
3                 0
4                 1
...             ...
5573              0
5574              0
5575              0
5576              0
5577              1

[5578 rows x 1 columns]


In [ ]:
def calculate_aromatic_proportion(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    aromatic_atoms = [atom for atom in mol.GetAtoms() if atom.GetIsAromatic()]
    total_atoms = mol.GetNumAtoms()
    if total_atoms == 0:
        return None
    return len(aromatic_atoms) / total_atoms

aromatic_proportion_data = []

for index, row in full.iterrows():
    smiles = row['SMILES']
    proportion = calculate_aromatic_proportion(smiles)

    if proportion is not None:
        aromatic_proportion_data.append({"aromatic_proportion": proportion})

aromatic_proportion_df = pd.DataFrame(aromatic_proportion_data)

print(aromatic_proportion_df)


      aromatic_proportion
0                0.571429
1                0.523810
2                0.590909
3                0.485714
4                0.600000
...                   ...
5573             0.625000
5574             0.000000
5575             0.200000
5576             0.480000
5577             0.454545

[5578 rows x 1 columns]


In [ ]:
def calculate_heavy_atom_count(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return mol.GetNumHeavyAtoms()

heavy_atom_data = []

for index, row in full.iterrows():
    smiles = row['SMILES']
    heavy_atom_count = calculate_heavy_atom_count(smiles)

    if heavy_atom_count is not None:
        heavy_atom_data.append({"heavy_atom_count": heavy_atom_count})

heavy_atom_df = pd.DataFrame(heavy_atom_data)

print(heavy_atom_df)


      heavy_atom_count
0                   28
1                   21
2                   22
3                   35
4                   20
...                ...
5573                24
5574                28
5575                30
5576                25
5577                33

[5578 rows x 1 columns]


In [ ]:
def calculate_formal_charge(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return Chem.rdmolops.GetFormalCharge(mol)

formal_charge_data = []

for index, row in full.iterrows():
    smiles = row['SMILES']
    charge = calculate_formal_charge(smiles)

    if charge is not None:
        formal_charge_data.append({"formal_charge": charge})

formal_charge_df = pd.DataFrame(formal_charge_data)

print(formal_charge_df)


      formal_charge
0                 0
1                 0
2                 0
3                 0
4                 0
...             ...
5573              0
5574              0
5575              0
5576              0
5577              0

[5578 rows x 1 columns]


In [ ]:
def calculate_rule_of_five_satisfactions(smiles, mol_MW, accptHB, donorHB):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None

    QPlogPo_w = Descriptors.MolLogP(mol)

    satisfactions = 0
    if mol_MW >= 500:
        satisfactions += 1
    if QPlogPo_w >= 5:
        satisfactions += 1
    if donorHB > 5:
        satisfactions += 1
    if accptHB > 10:
        satisfactions += 1

    return satisfactions, QPlogPo_w

rule_of_five_data = []

for index, row in full.iterrows():
    smiles = row['SMILES']
    mol_MW = row['Molecular_Weight']
    accptHB = row['Num_H_Acceptors']
    donorHB = row['Num_H_Donors']

    rule_of_five_satisfactions, QPlogPo_w_value = calculate_rule_of_five_satisfactions(smiles, mol_MW, accptHB, donorHB)

    if rule_of_five_satisfactions is not None:
        rule_of_five_data.append({
            "rule_of_five_satisfactions": rule_of_five_satisfactions,
            "QPlogPo_w": QPlogPo_w_value
        })

rule_of_five_df = pd.DataFrame(rule_of_five_data)

print(rule_of_five_df)


      rule_of_five_satisfactions  QPlogPo_w
0                              0    3.87744
1                              0    3.35474
2                              0    1.20450
3                              0    3.89356
4                              0    2.81772
...                          ...        ...
5573                           0    3.51482
5574                           1    5.24000
5575                           0    4.34280
5576                           0    1.94512
5577                           1    5.05290

[5578 rows x 2 columns]


In [ ]:
def calculate_molar_refractivity(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Descriptors.MolMR(mol)
    return None

def calculate_vdw_volume(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Descriptors.HeavyAtomCount(mol)
    return None

def calculate_molecular_flexibility(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return Descriptors.NumRotatableBonds(mol)
    return None

def calculate_electronegativity(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        total_eneg = sum([atom.GetAtomicNum() for atom in mol.GetAtoms()])
        return total_eneg
    return None

def calculate_pharmacophore_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return len([atom for atom in mol.GetAtoms() if atom.GetIsAromatic()])
    return None

data = []

for index, row in full.iterrows():
    smiles = row['SMILES']

    mr = calculate_molar_refractivity(smiles)
    vdw = calculate_vdw_volume(smiles)
    flex = calculate_molecular_flexibility(smiles)
    eneg = calculate_electronegativity(smiles)
    pharm = calculate_pharmacophore_features(smiles)

    data.append({"Molar Refractivity": mr,
                 "VdW Volume": vdw,
                 "Flexibility": flex,
                 "Electronegativity": eneg,
                 "Pharmacophore Features": pharm})

MVMEP_df = pd.DataFrame(data)

print(MVMEP_df)



      Molar Refractivity  VdW Volume  Flexibility  Electronegativity  \
0               109.1199          28            8                188   
1                86.8737          21            2                141   
2                85.0270          22            3                139   
3               136.0058          35            5                230   
4                77.1825          20            1                126   
...                  ...         ...          ...                ...   
5573             99.8419          24            5                149   
5574            111.2432          28           13                176   
5575            112.6437          30            5                190   
5576             89.9889          25            4                161   
5577            129.6223          33            8                229   

      Pharmacophore Features  
0                         16  
1                         11  
2                         13  
3          

In [ ]:
MVMEP_df.head(100)

,Molar Refractivity,VdW Volume,Flexibility,Electronegativity,Pharmacophore Features
0,109.1199,28,8,188,16
1,86.8737,21,2,141,11
2,85.0270,22,3,139,13
3,136.0058,35,5,230,17
4,77.1825,20,1,126,12
...,...,...,...,...,...
95,129.0760,34,9,230,18
96,160.9942,39,6,240,16
97,120.7370,31,4,237,18
98,94.3986,25,9,183,6


In [ ]:
from rdkit import Chem
from rdkit.Chem import rdmolops

fp_list = []
for idx, smiles in enumerate(full["SMILES"]): # Smiles 열에서 반복.
    mol = Chem.MolFromSmiles(smiles)
    fp = rdmolops.LayeredFingerprint(mol)
    fp_list.append(fp)

import sys
fp_bit_list = []
for fp in fp_list:
  fp_bit_list.append([int(x) for x in fp.ToBitString()])

dic = {}
for idx in range(2048):
  key = "F" + str(idx)
  dic[key] = []

for fp in fp_bit_list:
  for idx in range(2048):
    key = "F" + str(idx)
    dic[key].append(fp[idx])

Data_fp = pd.DataFrame(dic)

In [ ]:
from mordred import Calculator, descriptors

calc = Calculator(descriptors, ignore_3D=True)
len(calc.descriptors)

1613

In [ ]:
mord_desc_df = calc.pandas([Chem.MolFromSmiles(x) for x in full.SMILES])

  4%|▍         | 249/5578 [01:20<44:06,  2.01it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 21%|██        | 1173/5578 [06:01<40:44,  1.80it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 78%|███████▊  | 4336/5578 [22:35<07:27,  2.77it/s]

...

 98%|█████████▊| 5463/5578 [29:28<01:19,  1.45it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 99%|█████████▊| 5502/5578 [29:43<01:00,  1.25it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 5578/5578 [30:05<00:00,  3.09it/s]


In [ ]:
mord_desc_df.columns = ['mord_'+ column for column in mord_desc_df.columns]

In [ ]:
mord_desc_df.info()

<class 'mordred._base.pandas_module.MordredDataFrame'>
RangeIndex: 5578 entries, 0 to 5577
Columns: 1613 entries, mord_ABC to mord_mZagreb2
dtypes: bool(2), float64(683), int64(314), object(614)
memory usage: 68.6+ MB


In [ ]:
# object 형식 열 삭제
mord_desc_df.drop(columns=mord_desc_df.select_dtypes(include=['object']).columns, inplace=True)

# bool 형식 열을 0과 1로 인코딩
bool_columns = mord_desc_df.select_dtypes(include=['bool']).columns
mord_desc_df[bool_columns] = mord_desc_df[bool_columns].astype(int)

# 수정된 데이터프레임을 확인
print(mord_desc_df.head())

    mord_ABC  mord_ABCGG  mord_nAcid  mord_nBase  mord_nAromAtom  \
0  21.379612   17.449011           0           0              16   
1  16.539255   14.049653           0           0              11   
2  17.475469   13.660693           2           1              13   
3  27.857311   20.034364           0           1              17   
4  15.722758   12.817176           0           0              12   

   mord_nAromBond  mord_nAtom  mord_nHeavyAtom  mord_nSpiro  mord_nBridgehead  \
0              16          52               28            0                 0   
1              11          40               21            0                 0   
2              15          41               22            0                 0   
3              17          69               35            0                 0   
4              12          36               20            0                 0   

   ...  mord_SRW09  mord_SRW10  mord_TSRW10     mord_MW  mord_AMW  mord_WPath  \
0  ...    7.390799   10

In [ ]:
Data = pd.concat([full, full_Descrs, result_df, mord_desc_df, result_df2, result_df3, result_df4, NandO_df, ring_df, noncon_df, stereocenters_df, aromatic_proportion_df, heavy_atom_df, formal_charge_df, MVMEP_df, rule_of_five_df], axis=1).drop(columns=['Molecule', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])


In [ ]:
Data['beyond_Lipinski'] = 0

Data.loc[Data['Molecular_Weight'] >= 500, 'beyond_Lipinski'] += 1
Data.loc[Data['LogD'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_H_Donors'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_H_Acceptors'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_RotatableBonds'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[(Data['Num_H_Donors'] + Data['Num_H_Acceptors']) >= 12, 'beyond_Lipinski'] += 1
Data.loc[Data['TPSA'] >= 140, 'beyond_Lipinski'] += 1
Data.loc[Data['FractionCSP3'] <= 0.3, 'beyond_Lipinski'] += 1
Data.loc[Data['NumAromaticRings'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['NandO'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[Data['NHOHCount'] >= 5, 'beyond_Lipinski'] += 1

In [ ]:
Data["Molecular_PolarSurfaceArea"] = np.where(pd.isna(Data["Molecular_PolarSurfaceArea"]), Data["TPSA"], Data["Molecular_PolarSurfaceArea"])

In [ ]:
# 'id' 컬럼에 'TEST' 문자열이 들어가는 행을 testset으로 선택
test = Data[Data['id'].str.contains('TEST')]

# 'id' 컬럼에 'TEST' 문자열이 들어가지 않는 행을 trainset으로 선택
train = Data[~Data['id'].str.contains('TEST')]

In [ ]:
train

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,F158,F159,F160,F161,F162,F163,F164,F165,F166,beyond_Lipinski
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.25900,...,1,1,1,1,1,1,1,1,0,0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.17200,...,1,0,1,1,1,1,1,1,0,0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.58500,...,1,0,1,1,1,1,0,1,0,0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.47500,...,1,1,1,1,1,1,1,1,0,0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.33700,...,1,1,1,1,1,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5573,CHEMBL3286701,Cc1[nH]c2ccccc2c1CCNC(=O)c1ccc(N(C)C)cc1,NaN,17.000,3.510,321.420,2,2,5,3.51482,...,1,0,1,1,1,1,1,1,0,1
5574,CHEMBL215387,O=C(O)CCCCCCCCCCCNC(=O)NC12CC3CC(CC(C3)C1)C2,NaN,98.000,5.240,392.580,2,3,13,5.24000,...,1,1,0,1,0,1,1,1,0,2
5575,CHEMBL242459,O=C(NC12CC3CC(CC(C3)C1)C2)N[C@H]1CC[C@H](Oc2cc...,NaN,69.000,4.340,412.530,3,3,5,4.34280,...,1,1,0,1,1,1,1,1,0,0
5576,CHEMBL3735279,CCOC(=O)c1nc(C)c2c(c1N)C(=O)N(Cc1ccccc1)C2=O,NaN,14.000,1.950,339.350,6,1,4,1.94512,...,1,1,1,1,1,1,1,1,0,1


In [ ]:
test

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,F158,F159,F160,F161,F162,F163,F164,F165,F166,beyond_Lipinski
3498,TEST_000,CC(C)Nc1ccnc(N2CCN(Cc3cccs3)C(CCO)C2)n1,NaN,NaN,2.641,361.505,4,2,7,2.635,...,1,0,1,1,1,1,1,1,0,0
3499,TEST_001,COc1cc(=O)n(-c2ccccc2)cc1C(=O)N1CCC2(CC1)OCCO2,NaN,NaN,0.585,370.399,5,0,3,0.585,...,1,1,1,1,1,1,1,1,0,0
3500,TEST_002,Cc1cccc(NC(=N)/N=c2\nc(O)c(Cc3ccccc3)c(C)[nH]2)c1,NaN,NaN,4.276,347.414,4,4,5,4.290,...,1,0,1,1,1,1,1,1,0,1
3501,TEST_003,O=C(c1nc2ncccn2n1)N1CCCn2cc(-c3ccccc3)nc21,NaN,NaN,1.795,345.358,5,0,2,1.795,...,1,0,0,1,1,1,1,1,0,1
3502,TEST_004,CCN1CCN(C(=O)c2cc3c(=O)n4cc(C)ccc4nc3n2C)CC1,NaN,NaN,1.219,353.418,4,0,2,0.169,...,1,1,1,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3976,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,4.207,306.443,2,1,7,4.207,...,1,1,1,1,1,1,1,1,0,0
3977,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,-0.608,335.398,5,0,1,-1.736,...,1,1,1,1,0,1,1,1,0,0
3978,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,1.792,349.383,3,1,3,1.792,...,1,1,1,1,1,1,1,1,0,1
3979,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,0.790,341.132,3,2,2,0.423,...,1,1,1,1,1,1,1,1,0,1


In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# 숫자형식 컬럼들의 min-max 정규화
scaler = MinMaxScaler()
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()
non_features = ['id', 'SMILES', 'MLM', 'HLM','Fingerprint']
features = [column for column in train.columns if column not in non_features]
train[features] = scaler.fit_transform(train[features])

test[features] = scaler.transform(test[features])

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt

In [ ]:
duplicates = full[full.duplicated(subset='SMILES', keep=False)].sort_values(by='SMILES')

In [ ]:
duplicates

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
4668,CHEMBL570752,C#CC1CC2Cc3[nH]ncc3C(C1)N2S(=O)(=O)c1ccc(Cl)cc1,NaN,63.000,2.76,361.850,3,1,2,2.7630,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b5063760>
4324,CHEMBL570752,C#CC1CC2Cc3[nH]ncc3C(C1)N2S(=O)(=O)c1ccc(Cl)cc1,NaN,2.000,2.76,361.850,3,1,2,2.7630,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b5059fc0>
4453,CHEMBL2332218,C#Cc1ccc2c(c1)C(c1ccccc1)=NCc1c(-c3nc(C(C)C)no...,NaN,108.000,4.38,393.450,6,0,3,4.3780,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b505d8c0>
5185,CHEMBL2332218,C#Cc1ccc2c(c1)C(c1ccccc1)=NCc1c(-c3nc(C(C)C)no...,NaN,109.700,4.38,393.450,6,0,3,4.3780,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4e71bd0>
4154,CHEMBL2332215,C#Cc1ccc2c(c1)C(c1ccccc1)=NCc1c(C(=O)NC)ncn1-2,NaN,37.500,2.56,340.390,4,1,2,2.5642,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b50554d0>
...,...,...,...,...,...,...,...,...,...,...,...,...
5037,CHEMBL1585484,O=S(=O)(c1ccc2[nH]c(SCc3cccc(Cl)c3)nc2c1)N1CCOCC1,NaN,6.700,3.53,423.950,5,1,5,3.5295,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4e6da80>
5027,CHEMBL2396958,O=S(=O)(c1cccc(C(F)(F)F)c1)N1[C@@H]2c3ccccc3[C...,NaN,75.000,3.82,405.400,3,1,2,3.8196,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b4e6d620>
4644,CHEMBL2396958,O=S(=O)(c1cccc(C(F)(F)F)c1)N1[C@@H]2c3ccccc3[C...,NaN,47.000,3.82,405.400,3,1,2,3.8196,NaN,<rdkit.Chem.rdchem.Mol object at 0x7ab3b5062ce0>
2134,TRAIN_2134,O=S(=O)(c1ccccc1)n1ccc(C/C=C/Cc2ccn(S(=O)(=O)c...,1.416,1.404,5.63,466.572,4,0,8,5.6300,94.9,<rdkit.Chem.rdchem.Mol object at 0x7ab3b501d930>


In [ ]:
train_HLM = train.drop(columns=['MLM']).dropna(axis=0)

In [ ]:
train_HLM

,id,SMILES,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,F158,F159,F160,F161,F162,F163,F164,F165,F166,beyond_Lipinski
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,50.680,0.554964,0.192138,0.294118,0.1250,0.210526,0.579660,0.278787,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,50.590,0.473175,0.108751,0.117647,0.0625,0.052632,0.502622,0.171552,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,80.892,0.429954,0.105343,0.294118,0.0000,0.078947,0.461021,0.144633,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,2.000,0.668417,0.271376,0.352941,0.0000,0.131579,0.594968,0.218281,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,99.990,0.485631,0.080898,0.176471,0.0000,0.026316,0.514316,0.095730,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5573,CHEMBL3286701,Cc1[nH]c2ccccc2c1CCNC(=O)c1ccc(N(C)C)cc1,17.000,0.573798,0.125593,0.117647,0.1250,0.131579,0.597790,0.109654,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.1
5574,CHEMBL215387,O=C(O)CCCCCCCCCCCNC(=O)NC12CC3CC(CC(C3)C1)C2,98.000,0.703609,0.185477,0.117647,0.1875,0.342105,0.720057,0.183668,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.2
5575,CHEMBL242459,O=C(NC12CC3CC(CC(C3)C1)C2)N[C@H]1CC[C@H](Oc2cc...,69.000,0.636077,0.202266,0.176471,0.1875,0.131579,0.656471,0.206214,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
5576,CHEMBL3735279,CCOC(=O)c1nc(C)c2c(c1N)C(=O)N(Cc1ccccc1)C2=O,14.000,0.456742,0.140682,0.352941,0.0625,0.105263,0.486543,0.242684,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.1


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import math
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define features and targets
non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint']
features = [column for column in train.columns if column not in non_features]
mlm_target = "MLM"
hlm_target = "HLM"

# Initialize KFold
seed = 42
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

# Initialize arrays to store models and scores
reg_mlms = []
reg_hlms = []

# Initialize arrays to store RMSE scores
mlm_rmse_scores = []
hlm_rmse_scores = []

# Loop through KFold splits
for i, (train_index, valid_index) in enumerate(kf.split(train_HLM)):
    df_train = train_HLM.iloc[train_index]
    df_valid = train_HLM.iloc[valid_index]

    x_train_num = df_train[features].values

    y_hlm_train = df_train[hlm_target].values

    x_valid_num = df_valid[features].values

    y_hlm_valid = df_valid[hlm_target].values


    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(256, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    merged = layers.Dense(1024, activation='relu')(x1)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model_hlm = keras.Model(inputs=x1_input, outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model_hlm.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

    checkpoint_hlm = ModelCheckpoint(f'model_hlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_hlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model_hlm.fit(x_train_num, y_hlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_hlm_valid),
                  callbacks=[checkpoint_hlm, early_stopping_hlm])

    reg_hlms.append(model_hlm)

    # Calculate RMSE for HLM predictions
    y_hlm_pred = model_hlm.predict(x_valid_num)
    hlm_rmse = math.sqrt(mean_squared_error(y_hlm_valid, y_hlm_pred))
    print(hlm_rmse)
    hlm_rmse_scores.append(hlm_rmse)

Epoch 1/500
286/286 [==============================] - 2s 5ms/step - loss: 1345.2596 - root_mean_squared_error: 36.6778 - val_loss: 1031.2708 - val_root_mean_squared_error: 32.1134
Epoch 2/500
286/286 [==============================] - 1s 4ms/step - loss: 1016.5448 - root_mean_squared_error: 31.8833 - val_loss: 894.8367 - val_root_mean_squared_error: 29.9138
Epoch 3/500
286/286 [==============================] - 1s 4ms/step - loss: 961.2432 - root_mean_squared_error: 31.0039 - val_loss: 869.2242 - val_root_mean_squared_error: 29.4826
Epoch 4/500
286/286 [==============================] - 1s 4ms/step - loss: 914.3873 - root_mean_squared_error: 30.2388 - val_loss: 840.7510 - val_root_mean_squared_error: 28.9957
Epoch 5/500
286/286 [==============================] - 1s 4ms/step - loss: 886.5049 - root_mean_squared_error: 29.7742 - val_loss: 834.2382 - val_root_mean_squared_error: 28.8832
Epoch 6/500
286/286 [==============================] - 1s 4ms/step - loss: 863.3553 - root_mean_square

In [ ]:
train_MLM = train.dropna(axis=0)

In [ ]:
train_MLM

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,F158,F159,F160,F161,F162,F163,F164,F165,F166,beyond_Lipinski
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,0.554964,0.192138,0.294118,0.1250,0.210526,0.579660,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,0.473175,0.108751,0.117647,0.0625,0.052632,0.502622,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,0.429954,0.105343,0.294118,0.0000,0.078947,0.461021,...,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,0.668417,0.271376,0.352941,0.0000,0.131579,0.594968,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,0.485631,0.080898,0.176471,0.0000,0.026316,0.514316,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,TRAIN_3493,Cn1nc(CNC(=O)Cn2nc(C(F)(F)F)c3c2CCC3)c(Cl)c1Cl,1.556,3.079,0.566219,0.188519,0.176471,0.0625,0.131579,0.590291,...,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0
3494,TRAIN_3494,CCn1[nH]cc/c1=N\C(=O)c1nn(-c2ccccc2)c(=O)c2ccc...,35.560,47.630,0.453891,0.157539,0.235294,0.0625,0.078947,0.479376,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.1
3495,TRAIN_3495,CCOC(=O)CCCc1nc2cc(N)ccc2n1C,56.150,1.790,0.456067,0.075016,0.176471,0.0625,0.157895,0.499220,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
3496,TRAIN_3496,Nc1cc(C(=O)OCCC2CCOC2=O)cnc1Cl,0.030,2.770,0.384633,0.094688,0.294118,0.0625,0.131579,0.418781,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import math
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define features and targets
non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint']
features = [column for column in train.columns if column not in non_features]
mlm_target = "MLM"
hlm_target = "HLM"

# Initialize KFold
seed = 42
n_splits = 10
kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

# Initialize arrays to store models and scores
reg_mlms = []
reg_hlms = []

# Initialize arrays to store RMSE scores
mlm_rmse_scores = []
hlm_rmse_scores = []

# Loop through KFold splits
for i, (train_index, valid_index) in enumerate(kf.split(train_MLM)):
    df_train = train_MLM.iloc[train_index]
    df_valid = train_MLM.iloc[valid_index]

    x_train_num = df_train[features].values

    y_mlm_train = df_train[mlm_target].values

    x_valid_num = df_valid[features].values

    y_mlm_valid = df_valid[mlm_target].values

    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(256, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    merged = layers.Dense(1024, activation='relu')(x1)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model = keras.Model(inputs=x1_input, outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

    # Create and compile another model for HLM
    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(256, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    merged = layers.Dense(1024, activation='relu')(x1)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model_hlm = keras.Model(inputs=x1_input, outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model_hlm.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])


    # Train the model
    checkpoint_mlm = ModelCheckpoint(f'model_mlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_mlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model.fit(x_train_num, y_mlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_mlm_valid),
              callbacks=[checkpoint_mlm, early_stopping_mlm])

    reg_mlms.append(model)

    # checkpoint_hlm = ModelCheckpoint(f'model_hlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    # early_stopping_hlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # model_hlm.fit(x_train_num, y_hlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_hlm_valid),
    #               callbacks=[checkpoint_hlm, early_stopping_hlm])

    # reg_hlms.append(model_hlm)

    # Calculate RMSE for MLM predictions
    y_mlm_pred = model.predict(x_valid_num)
    mlm_rmse = math.sqrt(mean_squared_error(y_mlm_valid, y_mlm_pred))
    print(mlm_rmse)
    mlm_rmse_scores.append(mlm_rmse)

    # # Calculate RMSE for HLM predictions
    # y_hlm_pred = model_hlm.predict(x_valid_num)
    # hlm_rmse = math.sqrt(mean_squared_error(y_hlm_valid, y_hlm_pred))
    # print(hlm_rmse)
    # hlm_rmse_scores.append(hlm_rmse)

Epoch 1/500
197/197 [==============================] - 3s 7ms/step - loss: 1331.7977 - root_mean_squared_error: 36.4938 - val_loss: 1087.9697 - val_root_mean_squared_error: 32.9844
Epoch 2/500
197/197 [==============================] - 1s 5ms/step - loss: 1043.9402 - root_mean_squared_error: 32.3101 - val_loss: 955.7313 - val_root_mean_squared_error: 30.9149
Epoch 3/500
197/197 [==============================] - 1s 5ms/step - loss: 977.7197 - root_mean_squared_error: 31.2685 - val_loss: 901.8160 - val_root_mean_squared_error: 30.0303
Epoch 4/500
197/197 [==============================] - 1s 5ms/step - loss: 946.3215 - root_mean_squared_error: 30.7623 - val_loss: 890.7354 - val_root_mean_squared_error: 29.8452
Epoch 5/500
197/197 [==============================] - 1s 5ms/step - loss: 919.4837 - root_mean_squared_error: 30.3230 - val_loss: 873.0974 - val_root_mean_squared_error: 29.5482
Epoch 6/500
197/197 [==============================] - 1s 5ms/step - loss: 885.2442 - root_mean_square

In [ ]:
from keras.models import load_model

# Load MLM models
mlm_models = [load_model(f'model_mlm_fold{i}.h5') for i in range(n_splits)]

# Load HLM models
hlm_models = [load_model(f'model_hlm_fold{i}.h5') for i in range(n_splits)]

x_test = test[features].values

mlm_predictions = []
hlm_predictions = []

for model_mlm, model_hlm in zip(mlm_models, hlm_models):
    mlm_predictions.append(model_mlm.predict(x_test))
    hlm_predictions.append(model_hlm.predict(x_test))

# Convert prediction lists to numpy arrays
mlm_predictions = np.array(mlm_predictions)
hlm_predictions = np.array(hlm_predictions)

mlm_ensemble_prediction = mlm_predictions.mean(axis=0)
hlm_ensemble_prediction = hlm_predictions.mean(axis=0)

16/16 [==============================] - 0s 2ms/step


In [ ]:
df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")
df_submission["MLM"] = mlm_ensemble_prediction
df_submission["HLM"] = hlm_ensemble_prediction
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")

In [ ]:
df_submission

,id,MLM,HLM
0,TEST_000,24.310007,46.666012
1,TEST_001,69.935043,80.096207
2,TEST_002,40.984192,63.482807
3,TEST_003,44.150948,67.974350
4,TEST_004,59.096882,75.685013
...,...,...,...
478,TEST_478,9.344149,25.460550
479,TEST_479,76.723953,85.538841
480,TEST_480,48.334000,66.661972
481,TEST_481,49.098228,69.241417
